# RNN Welding

In [1]:
# 导入
import sys

sys.path.append("E:/dataFiles/github/MFlow")

In [2]:
# 数据生成
from data.generater import waveData

seq_len = 96  # 序列长度
in_dim = 16  # 输入维度
state_dim = 12  # 状态维度

xs, ys = waveData(1000, in_dim, seq_len)

train_xs = xs[:700]
train_ys = ys[:700]
test_xs = xs[700:]
test_ys = ys[700:]

print(train_xs.shape, train_ys.shape)
print(train_xs[0], train_ys[0])

(700, 96, 16) (700, 2)
[[ 0.58247709  0.7151034   0.81156869 ...  0.67007071  1.30002845
   1.86095641]
 [ 1.33161277  0.59030373  1.36051781 ...  1.57070144  0.04328647
   0.73381408]
 [ 1.1523317   0.78409598 -0.1004835  ...  0.8484807   0.32806824
   1.30682291]
 ...
 [-2.0003798  -1.13213328 -1.10660728 ... -2.10621726 -1.2449132
  -0.14521899]
 [-0.71458375 -1.21375394 -0.42217523 ... -0.79160263 -0.83155998
  -1.41592227]
 [-1.86114291 -1.80008492 -1.08304911 ... -0.3449521  -0.42370291
  -1.05659988]] [0. 1.]


In [3]:
# 训练
import numpy as np
from mflow import core, ops, opts, lays

# 超参数
lr = 0.005
epoch = 10  # 30
batch_size = 16

with core.NameScope("RNNWelding"):
    # 初始化变量
    inputs = [core.Variable(size=(in_dim, 1), trainable=False) for _ in range(seq_len)]
    u = core.Variable(size=(state_dim, in_dim), trainable=True)
    w = core.Variable(size=(state_dim, state_dim), trainable=True)
    b = core.Variable(size=(state_dim, 1), trainable=True)
    y = core.Variable(size=(2, 1), trainable=False)
    last_step = None
    hiddens = []
    # 网络构建
    for iv in inputs:
        h = ops.Add(ops.MatMul(u, iv), b)
        if last_step is not None:
            h = ops.Add(ops.MatMul(w, last_step), h)
        h = ops.ReLU(h)
        last_step = h
        hiddens.append(last_step)
    # 焊接点
    welding_point = ops.Welding()
    # 全连接网络
    fc_1 = lays.Linear(welding_point, state_dim, 40, "ReLU")
    fc_2 = lays.Linear(fc_1, 40, 10, "ReLU")
    pred = lays.Linear(fc_2, 10, 2, None)
    predicter = ops.Logistic(pred)
    loss = ops.loss.CrossEntropyWithSoftMax(pred, y)
    adam = opts.Adam(core.DefaultGraph, loss, lr)
    # 开始训练
    for ep in range(1, epoch + 1):
        bs_idx = 0  # 批次计数
        # 这是一个epoch的过程
        for i, (feat, lab) in enumerate(zip(train_xs, train_ys)):
            # 取变长序列
            lens = len(feat)
            start = np.random.randint(lens // 3)
            end = np.random.randint(lens // 3 + 30, lens)
            feat = feat[start: end]
            # 变长输入进入向量节点
            for j in range(len(feat)):
                inputs[j].setValue(np.mat(feat[j]).T)
            # 网络焊接
            welding_point.weld(hiddens[j])
            y.setValue(np.mat(lab).T)
            adam.step()
            bs_idx += 1
            if bs_idx == batch_size:
                if (i + 1) % 64 == 0:
                    print("Epoch: {:d}, itet: {:d}, loss: {:.7f}.".format(
                        ep, i + 1, loss.value[0, 0]))
                adam.update()
                bs_idx = 0
        # 一个epoch完成后进行评估
        preds = []
        for feat in test_xs:
            lens = len(feat)
            start = np.random.randint(lens // 3)
            end = np.random.randint(lens // 3 + 30, lens)
            feat = feat[start: end]
            for j in range(len(feat)):
                inputs[j].setValue(np.mat(feat[j]).T)
            welding_point.weld(hiddens[j])
            predicter.forward()
            preds.append(predicter.value.A.ravel())  # 结果
        preds = np.array(preds).argmax(axis=1)
        trues = test_ys.argmax(axis=1)
        acc = (trues == preds).astype("uint8").sum() / len(test_xs)
        print("Epoch: {:d}, acc: {:.3f}.".format(ep, acc))

Epoch: 1, itet: 64, loss: 0.6790788.
Epoch: 1, itet: 128, loss: 0.7156147.
Epoch: 1, itet: 192, loss: 0.7113709.
Epoch: 1, itet: 256, loss: 0.6943679.
Epoch: 1, itet: 320, loss: 0.6549510.
Epoch: 1, itet: 384, loss: 0.6311668.
Epoch: 1, itet: 448, loss: 0.5446169.
Epoch: 1, itet: 512, loss: 0.9997517.
Epoch: 1, itet: 576, loss: 0.2870387.
Epoch: 1, itet: 640, loss: 0.1704207.
Epoch: 1, acc: 0.643.
Epoch: 2, itet: 64, loss: 0.6695519.
Epoch: 2, itet: 128, loss: 0.5603998.
Epoch: 2, itet: 192, loss: 0.0061155.
Epoch: 2, itet: 256, loss: 0.0073545.
Epoch: 2, itet: 320, loss: 0.3604171.
Epoch: 2, itet: 384, loss: 0.4284815.
Epoch: 2, itet: 448, loss: 0.5525988.
Epoch: 2, itet: 512, loss: 1.0060855.
Epoch: 2, itet: 576, loss: 0.0902942.
Epoch: 2, itet: 640, loss: 0.2774523.
Epoch: 2, acc: 0.787.
Epoch: 3, itet: 64, loss: 0.2647876.
Epoch: 3, itet: 128, loss: 0.4772800.
Epoch: 3, itet: 192, loss: 0.0674703.
Epoch: 3, itet: 256, loss: 1.3197610.
Epoch: 3, itet: 320, loss: 0.4244872.
Epoch: 3,